* NAME: RISHABH PATIL
* SAP: 60009200056
* BATCH: K2

In [ ]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [ ]:
nltk.download('treebank')
nltk.download('universal_tagset')
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(nltk_data[:2])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [ ]:
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [ ]:
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [ ]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [ ]:
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [ ]:
tags = {tag for word,tag in train_tagged_words}
vocab = {word for word,tag in train_tagged_words}
print(len(tags))
print(tags)

12
{'NUM', 'ADV', 'X', '.', 'NOUN', 'PRON', 'DET', 'CONJ', 'ADP', 'ADJ', 'PRT', 'VERB'}


**Emission Probability**

In [ ]:
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

**Transition Probability**

In [ ]:
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[1.84219927e-01 3.57015361e-03 2.02427700e-01 1.19243130e-01
  3.51660132e-01 1.42806140e-03 3.57015361e-03 1.42806144e-02
  3.74866128e-02 3.53445187e-02 2.60621198e-02 2.07068902e-02]
 [2.98681147e-02 8.14584941e-02 2.28859577e-02 1.39255241e-01
  3.21955010e-02 1.20248254e-02 7.13731572e-02 6.98215654e-03
  1.19472459e-01 1.30721495e-01 1.47401085e-02 3.39022487e-01]
 [3.07514891e-03 2.57543717e-02 7.57255405e-02 1.60868734e-01
  6.16951771e-02 5.41995019e-02 5.68902567e-02 1.03786280e-02
  1.42225638e-01 1.76821072e-02 1.85085520e-01 2.06419379e-01]
 [7.82104954e-02 5.25694676e-02 2.56410260e-02 9.23720598e-02
  2.18538776e-01 6.87694475e-02 1.72191828e-01 6.00793920e-02
  9.29084867e-02 4.61323895e-02 2.78940029e-03 8.96899477e-02]
 [9.14395228e-03 1.68945398e-02 2.88252197e-02 2.40094051e-01
  2.62344331e-01 4.65906132e-03 1.31063312e-02 4.24540639e-02
  1.76826611e-01 1.25838192e-02 4.39345129e-02 1.49133503e-01]
 [6.83371304e-03 3.69020514e-02 8.83826911e-02 4.19134386e-02
  2

In [ ]:
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,NUM,ADV,X,.,NOUN,PRON,DET,CONJ,ADP,ADJ,PRT,VERB
NUM,0.184220,0.003570,0.202428,0.119243,0.351660,0.001428,0.003570,0.014281,0.037487,0.035345,0.026062,0.020707
ADV,0.029868,0.081458,0.022886,0.139255,0.032196,0.012025,0.071373,0.006982,0.119472,0.130721,0.014740,0.339022
X,0.003075,0.025754,0.075726,0.160869,0.061695,0.054200,0.056890,0.010379,0.142226,0.017682,0.185086,0.206419
.,0.078210,0.052569,0.025641,0.092372,0.218539,0.068769,0.172192,0.060079,0.092908,0.046132,0.002789,0.089690
NOUN,0.009144,0.016895,0.028825,0.240094,0.262344,0.004659,0.013106,0.042454,0.176827,0.012584,0.043935,0.149134
PRON,0.006834,0.036902,0.088383,0.041913,0.212756,0.006834,0.009567,0.005011,0.022323,0.070615,0.014123,0.484738
DET,0.022855,0.012074,0.045134,0.017393,0.635906,0.003306,0.006037,0.000431,0.009918,0.206411,0.000287,0.040247
CONJ,0.040615,0.057080,0.009330,0.035126,0.349067,0.060373,0.123491,0.000549,0.055982,0.113611,0.004391,0.150384
ADP,0.063275,0.014553,0.034548,0.038724,0.323589,0.069603,0.320931,0.001012,0.016958,0.107062,0.001266,0.008479
ADJ,0.021748,0.005243,0.020971,0.066019,0.696893,0.000194,0.005243,0.016893,0.080583,0.063301,0.011456,0.011456


In [ ]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
random.seed(1234) 
rndom = [random.randint(1,len(test_set)) for x in range(10)]

test_run = [test_set[i] for i in rndom]
test_run_base = [tup for sent in test_run for tup in sent]
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [ ]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  49.21722149848938
Viterbi Algorithm Accuracy:  93.77990430622009


In [ ]:
test_sent="Will can see Marry"
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_withoutRules)

[('Will', 'NUM'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NUM')]
